# Summarizing December King St. Pilot Data Interactively

In [66]:
# import modules
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas as pd
import pandas.io.sql as pandasql
import statistics
import datetime
import ipywidgets as widgets
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.widgets import GraphWidget
import plotly.figure_factory as ff
from IPython.html import widgets
from IPython.display import display, clear_output
import sys

In [11]:
# connect to database
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [12]:
# grab king_pilot.dash_daily_dev data
sql = pg.SQL("SELECT * from king_pilot.dash_daily_dev")
frame = pandasql.read_sql(sql, con)

In [16]:
frame

,street,direction,dt,day_type,category,period,tt
0,Dundas,Eastbound,2017-10-06,Weekday,Baseline,AM Peak,10.236496
1,Dundas,Eastbound,2017-10-09,Weekday,Excluded,AM Peak,7.732954
2,Dundas,Eastbound,2017-10-10,Weekday,Baseline,AM Peak,10.042032
3,Dundas,Eastbound,2017-10-11,Weekday,Baseline,AM Peak,10.824804
4,Dundas,Eastbound,2017-10-12,Weekday,Baseline,AM Peak,10.410758
5,Dundas,Eastbound,2017-10-13,Weekday,Baseline,AM Peak,9.636965
6,Dundas,Eastbound,2017-10-16,Weekday,Excluded,AM Peak,13.866233
7,Dundas,Eastbound,2017-10-17,Weekday,Excluded,AM Peak,15.240893
8,Dundas,Eastbound,2017-10-18,Weekday,Excluded,AM Peak,13.478941
9,Dundas,Eastbound,2017-10-19,Weekday,Excluded,AM Peak,13.939118


### The Dataframe

The table containing december street data will be made with a pandas dataframe. Firstly, an array is made containing the cells in rows that will classify the data, i.e. 'Dundas, Eastbound, from Bathurst to Jarvis'. This array is zipped into a list. 

### Grabbing Specified Data from the Dataframe
The following function produces data from the datarame according to input specifications.

In [17]:
def grab(street, direction, day_type, period, category):
    l = []
    for i in range(len(frame)):
        if category == 'Baseline':
            if frame.day_type.values[i] == day_type and frame.period.values[i] == period and frame.street.values[i] == street and frame.direction.values[i] == direction and frame.category.values[i] == 'Baseline':
                l.append(frame.tt.values[i])
        elif category == 'Pilot':
            if frame.day_type.values[i] == day_type and frame.period.values[i] == period and frame.street.values[i] == street and frame.direction.values[i] == direction and frame.category.values[i] == 'Pilot' and frame.dt.values[i] >= datetime.date(2017, 12, 1) and frame.dt.values[i] <= datetime.date(2017, 12, 31):
                l.append(frame.tt.values[i])        
    return statistics.mean(l)

grab('Dundas', 'Eastbound', 'Weekday', 'AM Peak', 'Baseline')

10.551674566686172

In [18]:
valuesarr = [['East-West Corridors', 'Dundas', 'Dundas', 'Queen', 'Queen', 'Richmond', 
              'Adelaide', 'Wellington', 'Front', 'Front', 'North-South Corridors',
              'Bathurst', 'Bathurst', 'Spadina', 'Spadina', 
              'University',  'University', 'Yonge', 'Yonge', 'Jarvis', 'Jarvis'],
             [' ', 'EB', 'WB', 'EB', 'WB', 'WB', 'EB',  
              'WB', 'EB', 'WB', '', 'NB', 'SB', 'NB', 'SB', 
              'NB', 'SB', 'NB', 'SB', 'NB', 'SB' ],
             [' ','Bathurst', 'Jarvis', 'Bathurst', 'Jarvis', 'Jarvis', 'Spadina',
             'Jarvis', 'Bathurst', 'Yonge', '', 'Front', 'Dundas', 'Front', 'Dundas',
             'Front', 'Dundas', 'Front', 'Dundas', 'Front', 'Dundas'],
             [' ','Jarvis', 'Bathurst', 'Jarvis', 'Bathurst', 'Bathurst', 'Jarvis', 'Blue Jays',
             'Jarvis', 'Bathurst', '', 'Dundas', 'Front', 'Dundas', 'Front', 'Dundas', 'Front', 
              'Dundas', 'Front', 'Dundas', 'Front']]
tuples = list(zip(*valuesarr))

The column names for the data classifiers are made via the MultiIndex method built in pandas.

In [93]:
index = pd.MultiIndex.from_tuples(tuples, names=['Street Name', 'Direction', 'From', 'To'])




dundas = [grab('Dundas', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
          grab('Dundas', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
dundas.append(dundas[0]-dundas[1])
dundas = dundas + [grab('Dundas', 'Westbound',  typew.value , periodw.value, 'Pilot'),
          grab('Dundas', 'Westbound',  typew.value , periodw.value, 'Baseline')]
dundas.append(dundas[3]-dundas[4])


queen = [grab('Queen', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
          grab('Queen', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
queen.append(queen[0]-queen[1])
queen = queen + [grab('Queen', 'Westbound',  typew.value , periodw.value, 'Pilot'),
          grab('Queen', 'Westbound',  typew.value , periodw.value, 'Baseline')]
queen.append(queen[3]-queen[4])


richmond = [grab('Richmond', 'Westbound',  typew.value , periodw.value, 'Pilot'),
          grab('Richmond', 'Westbound',  typew.value , periodw.value, 'Baseline')]
richmond.append(richmond[0]-richmond[1])


adelaide = [grab('Adelaide', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
          grab('Adelaide', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
adelaide.append(adelaide[0]-adelaide[1])


wellington = [grab('Wellington', 'Westbound',  typew.value , periodw.value, 'Pilot'),
          grab('Wellington', 'Westbound',  typew.value , periodw.value, 'Baseline')]
wellington.append(wellington[0]-wellington[1])


front = [grab('Front', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
          grab('Front', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
front.append(front[0]-front[1])
front = front + [grab('Front', 'Westbound',  typew.value , periodw.value, 'Pilot'),
          grab('Front', 'Westbound',  typew.value , periodw.value, 'Baseline')]
front.append(front[3]-front[4])


bathurst = [grab('Bathurst', 'Northbound', typew.value, periodw.value, 'Pilot'),
           grab('Bathurst', 'Northbound', typew.value, periodw.value, 'Baseline')]
bathurst.append(bathurst[0]-bathurst[1])
bathurst = bathurst + [grab('Bathurst', 'Southbound', typew.value, periodw.value, 'Pilot'),
                   grab('Bathurst', 'Southbound', typew.value, periodw.value, 'Baseline')]
bathurst.append(bathurst[3]-bathurst[4])


spadina = [grab('Spadina', 'Northbound', typew.value, periodw.value, 'Pilot'), 
          grab('Spadina', 'Northbound', typew.value, periodw.value, 'Baseline')]
spadina.append(spadina[0]-spadina[1])
spadina = spadina + [grab('Spadina', 'Southbound', typew.value, periodw.value, 'Pilot'),
                    grab('Spadina', 'Southbound', typew.value, periodw.value, 'Baseline')]
spadina.append(spadina[3]-spadina[4])


university = [grab('University', 'Northbound', typew.value, periodw.value, 'Pilot'), 
             grab('University', 'Northbound', typew.value, periodw.value, 'Baseline')]
university.append(university[0]-university[1])
university = university + [grab('University', 'Southbound', typew.value, periodw.value, 'Pilot'),
                           grab('University', 'Southbound', typew.value, periodw.value, 'Baseline')]
university.append(university[3]-university[4])


yonge = [grab('Yonge', 'Northbound', typew.value, periodw.value, 'Pilot'),
        grab('Yonge', 'Northbound', typew.value, periodw.value, 'Baseline')]
yonge.append(yonge[0]-yonge[1])
yonge = yonge + [grab('Yonge', 'Southbound', typew.value, periodw.value, 'Pilot'),
                grab('Yonge', 'Southbound', typew.value, periodw.value, 'Baseline')]
yonge.append(yonge[3]-yonge[4])


jarvis = [grab('Jarvis', 'Northbound', typew.value, periodw.value, 'Pilot'),
         grab('Jarvis', 'Northbound', typew.value, periodw.value, 'Baseline')]
jarvis.append(jarvis[0]-jarvis[1])
jarvis = jarvis + [grab('Jarvis', 'Southbound', typew.value, periodw.value, 'Pilot'),
         grab('Jarvis', 'Southbound', typew.value, periodw.value, 'Baseline')]
jarvis.append(jarvis[3]-jarvis[4])



x = [[' ',' ',' ']]+[dundas[0:3],dundas[3:6],
                     queen[0:3], queen[3:6],
                     richmond[0:3],
                     adelaide[0:3], wellington[0:3],
                     front[0:3], front[3:6]] + [['','','']] + [bathurst[0:3], bathurst[3:6],
                     spadina[0:3], spadina[3:6],
                     university[0:3], university[3:6], 
                     yonge[0:3], yonge[3:6], 
                     jarvis[0:3], jarvis[3:6]]


df = pd.DataFrame(x, index=index,
                  columns=['Pilot', 'Baseline', 'Change'])

df

Pilot Baseline    Change
Street Name           Direction From     To                                   
East-West Corridors                                                           
Dundas                EB        Bathurst Jarvis     11.3524  11.8121 -0.459765
                      WB        Jarvis   Bathurst   11.5654  12.2198 -0.654483
Queen                 EB        Bathurst Jarvis     13.1289  11.9102   1.21876
                      WB        Jarvis   Bathurst   12.1125  11.5708  0.541658
Richmond              WB        Jarvis   Bathurst   8.61561  8.34095  0.274657
Adelaide              EB        Spadina  Jarvis      8.6198   8.1847  0.435093
Wellington            WB        Jarvis   Blue Jays  7.50505  7.38043  0.124624
Front                 EB        Bathurst Jarvis     11.2144  11.5715  -0.35702
                      WB        Yonge    Bathurst   10.4933  10.3203   0.17295
North-South Corridors                                                         
Bathurst              NB        Front    Dundas     5.22559    5.498 -0.272416
                      SB        Dundas   Front      4.55337  5.27467 -0.721303
Spadina               NB        Front    Dundas     6.28045  6.45209 -0.171635
                      SB        Dundas   Front      6.12473  6.50167 -0.376946
University            NB        Front    Dundas     4.91153   4.6786  0.232923
                      SB        Dundas   Front       4.6506  4.28781   0.36279
Yonge                 NB        Front    Dundas     6.64187  6.79955 -0.157684
                      SB        Dundas   Front      6.31823   7.1907 -0.872469
Jarvis                NB        Front    Dundas     4.29221  4.48722 -0.195005
                      SB        Dundas   Front      3.55888  3.89114 -0.332254

### Dropdown Widgets

The following are dropdown widgets with daytypes and periods. Since periods depend on daytypes, the options in the period dropdown will change depending on whether 'Weekday' or 'Weekend'. 

In [94]:
dtype={'Weekday':['AM Peak','Midday','PM Peak', 'Evening'],'Weekend':['Morning','Afternoon','Evening']}

def returnperiod(period):
    return period

def select_period(daytype):
    periodw.options = dtype[daytype]
    

button = widgets.Button(description="Generate Summary")


typew = widgets.Dropdown(options=dtype.keys(), description = 'Day Type')
periodw = widgets.Dropdown(options=dtype[typew.value], description = 'Period')

i = widgets.interactive(select_period, daytype=typew)
j = widgets.interactive(returnperiod, period=periodw)


def on_value_change(change):
    typew.value = change['new']
    periodw.value = change['new']

button.observe(on_value_change, names='value')

def ret(df):
    return(df)

out = widgets.Output()

def on_button_clicked(b):
    index = pd.MultiIndex.from_tuples(tuples, names=['Street Name', 'Direction', 'From', 'To'])
    
    dundas = [grab('Dundas', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
              grab('Dundas', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
    dundas.append(dundas[0]-dundas[1])
    dundas = dundas + [grab('Dundas', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Dundas', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    dundas.append(dundas[3]-dundas[4])
    queen = [grab('Queen', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
              grab('Queen', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
    queen.append(queen[0]-queen[1])
    queen = queen + [grab('Queen', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Queen', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    queen.append(queen[3]-queen[4])
    richmond = [grab('Richmond', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Richmond', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    richmond.append(richmond[0]-richmond[1])
    adelaide = [grab('Adelaide', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
              grab('Adelaide', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
    adelaide.append(adelaide[0]-adelaide[1])
    wellington = [grab('Wellington', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Wellington', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    wellington.append(wellington[0]-wellington[1])
    front = [grab('Front', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
              grab('Front', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
    front.append(front[0]-front[1])
    front = front + [grab('Front', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Front', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    front.append(front[3]-front[4])
    
    bathurst = [grab('Bathurst', 'Northbound', typew.value, periodw.value, 'Pilot'),
           grab('Bathurst', 'Northbound', typew.value, periodw.value, 'Baseline')]
    bathurst.append(bathurst[0]-bathurst[1])
    bathurst = bathurst + [grab('Bathurst', 'Southbound', typew.value, periodw.value, 'Pilot'),
                       grab('Bathurst', 'Southbound', typew.value, periodw.value, 'Baseline')]
    bathurst.append(bathurst[3]-bathurst[4])


    spadina = [grab('Spadina', 'Northbound', typew.value, periodw.value, 'Pilot'), 
              grab('Spadina', 'Northbound', typew.value, periodw.value, 'Baseline')]
    spadina.append(spadina[0]-spadina[1])
    spadina = spadina + [grab('Spadina', 'Southbound', typew.value, periodw.value, 'Pilot'),
                        grab('Spadina', 'Southbound', typew.value, periodw.value, 'Baseline')]
    spadina.append(spadina[3]-spadina[4])


    university = [grab('University', 'Northbound', typew.value, periodw.value, 'Pilot'), 
                 grab('University', 'Northbound', typew.value, periodw.value, 'Baseline')]
    university.append(university[0]-university[1])
    university = university + [grab('University', 'Southbound', typew.value, periodw.value, 'Pilot'),
                               grab('University', 'Southbound', typew.value, periodw.value, 'Baseline')]
    university.append(university[3]-university[4])


    yonge = [grab('Yonge', 'Northbound', typew.value, periodw.value, 'Pilot'),
            grab('Yonge', 'Northbound', typew.value, periodw.value, 'Baseline')]
    yonge.append(yonge[0]-yonge[1])
    yonge = yonge + [grab('Yonge', 'Southbound', typew.value, periodw.value, 'Pilot'),
                    grab('Yonge', 'Southbound', typew.value, periodw.value, 'Baseline')]
    yonge.append(yonge[3]-yonge[4])


    jarvis = [grab('Jarvis', 'Northbound', typew.value, periodw.value, 'Pilot'),
             grab('Jarvis', 'Northbound', typew.value, periodw.value, 'Baseline')]
    jarvis.append(jarvis[0]-jarvis[1])
    jarvis = jarvis + [grab('Jarvis', 'Southbound', typew.value, periodw.value, 'Pilot'),
             grab('Jarvis', 'Southbound', typew.value, periodw.value, 'Baseline')]
    jarvis.append(jarvis[3]-jarvis[4])

    
    x = [[' ',' ',' ']]+[dundas[0:3],dundas[3:6],
                     queen[0:3], queen[3:6],
                     richmond[0:3],
                     adelaide[0:3], wellington[0:3],
                     front[0:3], front[3:6]] + [['','','']] + [bathurst[0:3], bathurst[3:6],
                     spadina[0:3], spadina[3:6],
                     university[0:3], university[3:6], 
                     yonge[0:3], yonge[3:6], 
                     jarvis[0:3], jarvis[3:6]]
    
    df = pd.DataFrame(x, index=index,
                      columns=['Pilot', 'Baseline', 'Change'])
    with out:
        clear_output(True)
        display(df)
    
    #print(typew.value, periodw.value)
    

button.on_click(on_button_clicked)

display(i)
display(j)
display(button)

with out:
    display(df)
    
out


interactive(children=(Dropdown(description='Day Type', options=('Weekday', 'Weekend'), value='Weekday'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Dropdown(description='Period', options=('AM Peak', 'Midday', 'PM Peak', 'Evening'), value='AM Peak'), Output()), _dom_classes=('widget-interact',))

Button(description='Generate Summary', style=ButtonStyle())

Output()